# 🧪 Lab Task: Benchmarking Vector Indexing and Retrieval Methods

## 🎯 Objective
In this lab, you will **experiment with different vector indexing methods** and **measure their performance** for document retrieval using embedding-based search.

You will:
1. Generate text embeddings using a **Sentence Transformer** model.
2. Index the embeddings using **two approximate nearest neighbor (ANN)** methods.
3. Measure and compare **indexing time**, **retrieval latency**, and **retrieval quality**.

---

## 🧩 Background

In Retrieval-Augmented Generation (RAG) systems, a **vector database** is used to efficiently find semantically similar documents to a query.  
Different indexing methods (e.g., **Flat**, **HNSW**, **IVF**) trade off between **speed**, **accuracy**, and **memory usage**.

Understanding these trade-offs helps you design RAG systems that are **efficient** and **scalable**.

---

## ⚙️ Task Description

### Step 1. Prepare the data
- Create a small **corpus of text documents** (e.g., 1000–5000 Wikipedia sentences, news snippets, or sample text paragraphs).
- Create a separate list of **query sentences** for evaluation (e.g., 10–20 queries).

### Step 2. Generate embeddings
- Use a **Sentence Transformer** model (e.g., `"all-MiniLM-L6-v2"` or any other from [https://huggingface.co/sentence-transformers/models](https://huggingface.co/sentence-transformers/models)).
- Encode all documents and queries into embedding vectors.

```python
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(docs, normalize_embeddings=True)
query_embeddings = model.encode(queries, normalize_embeddings=True)
```

# Step 3. Build Indexes

You will build two different FAISS indexes and benchmark them:

## 🔹 Index 1: Flat (Exact Search)

* Use `faiss.IndexFlatL2` (exact nearest neighbor search).
* **Pros:** Perfect recall (most accurate).
* **Cons:** Slow for large datasets.

## 🔹 Index 2: HNSW (Approximate Search)

* Use `faiss.IndexHNSWFlat`.
* Set appropriate parameters (e.g., `M=32`, `efConstruction=100`).
* **Pros:** Much faster search; small loss in recall.

### Example setup:
```python
import faiss
import numpy as np
import time

dim = doc_embeddings.shape[1]

# Flat index
index_flat = faiss.IndexFlatL2(dim)

# HNSW index
index_hnsw = faiss.IndexHNSWFlat(dim, 32)
index_hnsw.hnsw.efConstruction = 100
index_hnsw.hnsw.efSearch = 50
```

# Step 4. Benchmarking

Measure and compare the following metrics:

| Metric | Description |
|--------|-------------|
| **Indexing Time** | Time to add all embeddings to the index. |
| **Retrieval Latency** | Average time to find top-k results for each query. |
| **Recall@k** | How often the retrieved top-k includes the true nearest neighbor. |
| **Memory Usage** (optional) | Approximate RAM usage of each index type. |

You can record times using `time.time()` or `time.perf_counter()`.


```markdown
## 💡 Tips

* Normalize embeddings before indexing for stable distance comparisons.
* Use `faiss.normalize_L2()` if needed.
* Experiment with `efSearch` and `M` parameters in HNSW to see how they impact performance.
``


In [1]:
import time
import math
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [2]:
text_path = "assets/text.txt"
queries_path = "assets/queries.txt"
text = []
queries = []

with open(text_path, "r", encoding="utf-8") as f:
    for line in f:
        s = line.strip()
        if s:
            text.append(s)

with open(queries_path, "r", encoding="utf-8") as f:
    for line in f:
        s = line.strip()
        if s:
            queries.append(s)

if len(text) == 0 or len(queries) == 0:
    raise ValueError("Either text or queries are not present")

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [4]:
text_embeddings = model.encode(text, normalize_embeddings=True)

dim = text_embeddings.shape[1]
ntotal = text_embeddings.shape[0]

num_queries = 10
rng = np.random.default_rng(0)
query_idx = rng.choice(ntotal, size=num_queries, replace=False)
query_embeddings = text_embeddings[query_idx]

In [5]:
faiss.normalize_L2(text_embeddings)
faiss.normalize_L2(query_embeddings)

In [6]:
# Flat index

index_flat = faiss.IndexFlatL2(dim)
t0 = time.perf_counter()
index_flat.add(text_embeddings)
t_flat = time.perf_counter() - t0

print(f"Flat index built in {t_flat:.3f}s")

Flat index built in 0.002s


In [7]:
# HNSW index

index_hnsw = faiss.IndexHNSWFlat(dim, 32)
index_hnsw.hnsw.efConstruction = 100
t0 = time.perf_counter()
index_hnsw.add(text_embeddings)
t_hnsw = time.perf_counter() - t0

print(f"HNSW index built in {t_hnsw:.3f}s")

HNSW index built in 0.172s


In [8]:
# Flat search
_ = index_flat.search(query_embeddings[:min(8, len(query_embeddings))], 1)
t0 = time.perf_counter()
D_true, I_true = index_flat.search(query_embeddings, 1)
avg_ms_flat = (time.perf_counter() - t0) / num_queries * 1000.0
true_nn = I_true[:, 0]

print(f"Flat index search time per query: {avg_ms_flat:.2f} ms")

Flat index search time per query: 0.82 ms


In [9]:
def test_hnsw_for_k(M, ef_construction, ef_search, k):
    index_hnsw = faiss.IndexHNSWFlat(dim, M)
    index_hnsw.hnsw.efConstruction = ef_construction

    t0 = time.perf_counter()
    index_hnsw.add(text_embeddings)
    build_s = time.perf_counter() - t0

    index_hnsw.hnsw.efSearch = ef_search
    _ = index_hnsw.search(query_embeddings[:min(8, len(query_embeddings))], k)
    t0 = time.perf_counter()
    D_h, I_h = index_hnsw.search(query_embeddings, k)
    latency_ms = (time.perf_counter() - t0) / num_queries * 1000.0

    hits = 0
    for i in range(len(true_nn)):
        if true_nn[i] in I_h[i, :k]:
            hits += 1
    recall_atk = hits / len(true_nn)

    mem_flat_bytes = ntotal * dim * 4
    mem_links_bytes = ntotal * M * 8
    approx_mem = mem_flat_bytes + mem_links_bytes

    return build_s, latency_ms, recall_atk, approx_mem

In [10]:
top_k_values = [1, 2, 5]
M_values = [4, 8, 16, 32]
ef_construction = 100
ef_search_values = [4, 8, 32, 100]

def human_bytes(n):
    units = ["B", "KB", "MB", "GB", "TB"]
    if n <= 0:
        return "0 B"
    i = int(math.floor(math.log(n, 1024)))
    p = 1024 ** i
    return f"{n / p:.2f} {units[i]}"

for k in top_k_values:
    print(f"\n=== HNSW sweep results for k = {k} ===\n")
    print("M   efSearch  build_s  latency_ms  recall@k  approx_RAM")

    results_k = []

    for M in M_values:
        for ef in ef_search_values:
            build_s, latency_ms, recall_atk, ram = test_hnsw_for_k(M, ef_construction, ef, k)
            results_k.append((M, ef, build_s, latency_ms, recall_atk, ram))
            print(
                f"{M:>2} {ef:>8}   "
                f"{build_s:.3f}     "
                f"{latency_ms:.2f}        "
                f"{recall_atk:.3f}      "
                f"{human_bytes(ram)}"
            )

    best = sorted(results_k, key=lambda r: (-r[4], r[3]))[0]
    print("\nSuggested setting for this k")
    print(
        f"M={best[0]}, efSearch={best[1]}, "
        f"recall@{k}={best[4]:.3f}, latency={best[3]:.2f} ms, "
        f"build_s={best[2]:.2f}, RAM={human_bytes(best[5])}"
    )



=== HNSW sweep results for k = 1 ===

M   efSearch  build_s  latency_ms  recall@k  approx_RAM
 4        4   0.135     0.00        0.500      6.64 MB
 4        8   0.152     0.01        0.500      6.64 MB
 4       32   0.145     0.01        0.600      6.64 MB
 4      100   0.157     0.04        0.800      6.64 MB
 8        4   0.182     0.01        0.600      6.77 MB
 8        8   0.189     0.01        0.600      6.77 MB
 8       32   0.216     0.01        0.500      6.77 MB
 8      100   0.186     0.03        0.500      6.77 MB
16        4   0.176     0.01        0.600      7.05 MB
16        8   0.185     0.01        0.700      7.05 MB
16       32   0.173     0.02        0.800      7.05 MB
16      100   0.194     0.03        0.500      7.05 MB
32        4   0.177     0.01        0.400      7.59 MB
32        8   0.178     0.01        0.500      7.59 MB
32       32   0.177     0.02        0.300      7.59 MB
32      100   0.174     0.04        0.800      7.59 MB

Suggested setting for th